### Importación de Librerias

In [1]:
import pandas as pd
import pickle
import warnings

warnings.filterwarnings('ignore')

# Configurar pandas para que no trunque la visualización
pd.set_option("display.max_columns", None)  # Mostrar todas las columnas
pd.set_option("display.max_colwidth", None)  # Mostrar todo el contenido de las celdas, si es posible
pd.set_option('display.max_rows', None) # Mostrar todos las filas

### Predictor

In [2]:
# Función para predecir los resultados de la carrera 
def predictor_grupos(model_path, df):
    """
    Cargamos el modelo guardado anteriormente para predecir el grupo de Posicion de los corredores de F1.

    Parámetros:
    - model_path: Ruta al archivo del modelo guardado
    - df: Un DataFrame que contiene las siguientes columnas:
      ['Race', 'Driver', 'Constructor', 'Circuit', 'Grid', 'Laps', 'Rank', 'Year', 'Date', 'Position_Order']

    Retorna:
    - Un diccionario de DataFrames con las predicciones del grupo de Posicion y la descripción del grupo por carrera.
    """
    # Cargar el modelo desde el archivo
    with open(model_path, 'rb') as file:
        model = pickle.load(file)
    
    # Verificar las columnas necesarias del DataFrame
    columnas_requeridas = ['Race', 'Driver', 'Constructor', 'Circuit', 'Grid', 'Laps', 'Rank', 'Year', 'Date', 'Position_Order']
    for col in columnas_requeridas:
        if col not in df.columns:
            raise ValueError(f"La columna {col} falta en el DataFrame de entrada.")
    
    # Definir la descripción de los grupos de Posicion
    position_desc = {
        1: "Posicion 1-3",
        2: "Posicion 4-6",
        3: "Posicion 7-9",
        4: "Posicion 10-14",
        5: "Posicion 15+"
    }
    
    # Crear un diccionario para almacenar los DataFrames de resultados por carrera
    resultados_por_carrera = {}
    
    # Agrupar el DataFrame por la columna 'Race'
    carreras = df['Race'].unique()
    
    for carrera in carreras:
        df_carrera = df[df['Race'] == carrera].copy()
        
        # Realizar las predicciones para la carrera actual
        predictions = model.predict(df_carrera)
        
        # Añadir las predicciones y la descripción al DataFrame de la carrera
        df_carrera['Position_Group'] = predictions
        df_carrera['Group'] = df_carrera['Position_Group'].map(position_desc)
        
        # Determinar si la predicción fue correcta
        df_carrera['Prediction'] = df_carrera.apply(
            lambda row: "Si" if (
                (row['Position_Order'] <= 3 and row['Group'] == "Posicion 1-3") or
                (4 <= row['Position_Order'] <= 6 and row['Group'] == "Posicion 4-6") or
                (7 <= row['Position_Order'] <= 9 and row['Group'] == "Posicion 7-9") or
                (10 <= row['Position_Order'] <= 14 and row['Group'] == "Posicion 10-14") or
                (row['Position_Order'] >= 15 and row['Group'] == "Posicion 15+")
            ) else "No",
            axis=1
        )
        
        # Contar cuántas de las 20 posiciones fueron correctamente predichas
        aciertos = df_carrera[df_carrera['Position_Order'] <= 20]['Prediction'].value_counts().get("Si", 0)
        
        # Seleccionar las columnas de interés
        result_df = df_carrera[['Driver', 'Position_Order', 'Group', 'Prediction']]
        
        # Guardar el DataFrame de resultados en el diccionario, incluyendo el conteo de aciertos
        resultados_por_carrera[carrera] = {
            'result_df': result_df,
            'aciertos': aciertos
        }
        
        # Guardar el DataFrame resultante en un archivo CSV
        #result_df.to_csv(f'../Resultados_Predicciones/resultados_{carrera}.csv', index=False)
    
    return resultados_por_carrera

In [3]:
# Ruta al modelo guardado
model_path = '../Models/best_rf_model.pkl'
# Leer el archivo CSV de prueba
df = pd.read_csv('../Test/Test_F1.csv')

resultados = predictor_grupos(model_path, df)

# Mostrar los resultados
for carrera, resultado in resultados.items():
    print(f"Resultados para la carrera: {carrera}")
    print(resultado['result_df'])
    print(f"Aciertos: {resultado['aciertos']}")

Resultados para la carrera: Dutch Grand Prix
             Driver  Position_Order           Group Prediction
0    Max Verstappen               1    Posicion 1-3         Si
1   Fernando Alonso               2    Posicion 4-6         No
2      Pierre Gasly               3  Posicion 10-14         No
3      Sergio Perez               4    Posicion 4-6         Si
4      Carlos Sainz               5    Posicion 7-9         No
5    Lewis Hamilton               6    Posicion 4-6         Si
6      Lando Norris               7    Posicion 1-3         No
7   Alexander Albon               8  Posicion 10-14         No
8     Oscar Piastri               9    Posicion 7-9         Si
9      Esteban Ocon              10  Posicion 10-14         Si
10     Lance Stroll              11  Posicion 10-14         Si
11  Nico Hulkenberg              12  Posicion 10-14         Si
12      Liam Lawson              13  Posicion 10-14         Si
13  Valtteri Bottas              14  Posicion 10-14         Si
14     Yuk